In [3]:
import tensorflow as tf
from tensorflow.keras import layers,losses
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D,Conv2D,Convolution2D,Activation,MaxPool2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
import keras
from keras import regularizers
from keras.layers import BatchNormalization
import numpy as np

In [4]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xqBmiS8Usq8MTMmmHGrR1TkLqXjc_eYMmOE9OtiuZYov',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'gesturebasedtoolforsterilebrowsin-donotdelete-pr-whvaawvll80t5c'
object_key = 'Finger Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [5]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
filepaths=unzip.namelist()
for path in filepaths:
    unzip.extract(path)

In [6]:
train_datagen = ImageDataGenerator( rescale=1./255,
                                    rotation_range=10.,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    zoom_range=0.2,
                                    horizontal_flip=True
                                  )
train_gen = train_datagen.flow_from_directory(
        r'/home/wsuser/work/Finger Dataset/train',
        target_size=(128,128),
        color_mode='grayscale',
        batch_size=32,
        classes=['0','1','2','3','4','5'],
        class_mode='categorical'
    )

Found 18000 images belonging to 6 classes.


In [7]:
test_datagen  = ImageDataGenerator( rescale=1./255 )
test_gen = test_datagen.flow_from_directory(
        r'/home/wsuser/work/Finger Dataset/test',
        target_size=(128,128),
        color_mode='grayscale',
        batch_size=32,
        classes=['0','1','2','3','4','5'],
        class_mode='categorical'
    )

Found 3600 images belonging to 6 classes.


In [8]:
model=Sequential()
model.add(BatchNormalization(input_shape = (128,128,1)))
model.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 1))) 
model.add(MaxPooling2D(pool_size=2))
model.add(Convolution2D(filters=6,kernel_size=4,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Convolution2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Convolution2D(filters=128,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten()) 
model.add(Dense(units=128,activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 6, activation = 'softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 128, 128, 1)      4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 63, 63, 6)         3078      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 31, 6)        0         
 2D)                                                             
                                                        

In [10]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [11]:
history=model.fit_generator(train_gen,
                    epochs=20,
                    steps_per_epoch=18000//32,
                    validation_data=test_gen,
                    verbose = 1,validation_steps=3600//32)

/tmp/wsuser/ipykernel_164/1240802224.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_gen,


Epoch 1/20
562/562 [==============================] - 308s 546ms/step - loss: 0.2178 - accuracy: 0.9182 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 2/20
562/562 [==============================] - 306s 545ms/step - loss: 0.0386 - accuracy: 0.9880 - val_loss: 0.0068 - val_accuracy: 0.9978
Epoch 3/20
562/562 [==============================] - 299s 532ms/step - loss: 0.0192 - accuracy: 0.9948 - val_loss: 7.7184e-04 - val_accuracy: 1.0000
Epoch 4/20
562/562 [==============================] - 306s 545ms/step - loss: 0.0137 - accuracy: 0.9960 - val_loss: 0.0032 - val_accuracy: 0.9983
Epoch 5/20
562/562 [==============================] - 310s 552ms/step - loss: 0.0184 - accuracy: 0.9940 - val_loss: 5.2546e-04 - val_accuracy: 1.0000
Epoch 6/20
562/562 [==============================] - 312s 556ms/step - loss: 0.0150 - accuracy: 0.9949 - val_loss: 1.4421e-05 - val_accuracy: 1.0000
Epoch 7/20
562/562 [==============================] - 304s 541ms/step - loss: 0.0073 - accuracy: 0.9978 - val_lo

In [19]:
model.save('gesture.h5')

In [12]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":'https://us-south.ml.cloud.ibm.com',
    "apikey":'on6wVLLy-ERS74JlvyDrFdJ35GRaHzaCtKxejqR7euwG'
}
client=APIClient(wml_credentials)

In [13]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
--  ----  -------
ID  NAME  CREATED
--  ----  -------


In [14]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [15]:
space_uid=guid_from_space_name(client,'Gesture_Deploy')
print('Space UID = '+space_uid)

Space UID = 2711769b-22de-47b3-8ece-f8eaeee3c1b6


In [16]:
client.set.default_space(space_uid)

'SUCCESS'

In [17]:
software_spec_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [20]:
!tar -zcvf gesture_based_tool.tgz gesture.h5

gesture.h5


In [22]:
model_details=client.repository.store_model(model='gesture_based_tool.tgz',meta_props={
                                             client.repository.ModelMetaNames.NAME:"Gesture Based Tool",
                                             client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
                                             client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid 
                                             }
                                           )
model_id=client.repository.get_model_id(model_details)

In [23]:
model_id

'ba7b8922-d173-42c1-9247-f7de568dff37'